<a href="https://colab.research.google.com/github/abyanahmadn/Dicoding-NLP-Model-TensorFlow/blob/main/Dicoding_P1_NLP_Model_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Dicoding ML Intermediate Project 1**

**Title: Create NLP Model with TensorFlow**

**Identity:**
* Name: Abyan Ahmad Nurrasyid
* Email: abyanahmadn@gmail.com